Week 1: Tic-Tac-Toe AI

This week we will be creating the ultimate Tic-Tac-Toe player. You are to code an AI of any form which will learn how to proficiently play the game. The AI will be pitted against each other in a competetion using this notebook as the host server.

Requirements:

1. Your player must actually learn how to play the game using machine learning AI, not be rule based AI.

2. While in the future this may change, a board is currently a 3x3 array of numbers 0-2 inclusive representing the states of each of the spots on the board. Your player must change the position it wants to move onto in the board to be the number in a variable called color. In our case, 1 will represent red aka x and 2 will represent blue aka o. 0 just means blank. Make sure the AI only changes spots on the board to the number 1 if color is 1 or the number 2 if color is 2.

3. Your player must interact properly with this notebook in order to participate in this competition.

Competition Rules:

Each person's AI will play with every other AI. Everyone will add up there total points from each match. Whoever has the most points in total by the end wins.

In each individual matchup the AI play two hundred thousand games between them. There will be a set of a hundred thousand games with player 1 going first, and then a hundred thousand with player two going first. This won't take as long as you think since only the first and last 10 games of each set will be shown visually. A player gets 2 points for a win, 1 for a tie, and 0 for a loss. The players will add the points to their total.

# Utilities

In [ ]:
#pretty print function and other libraries
# This funtion visualizes game states. It should run correctly along with your code as long as you give it the right input. This method takes in a board, a 3x3 array of integers from 0 to 2 inclusive. A zero means neither x nor o, blank, and will display white. A one means x and will display red. A two means o and will display blue.

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np

def pretty_print(board):
  fig, ax = plt.subplots()
  cmap = colors.ListedColormap(['white', 'red', 'blue'])
  bounds = [-0.5,0.5,1.5,2.5]
  norm = colors.BoundaryNorm(bounds, cmap.N)

  ax.imshow(np.array(board).reshape((3,3)), cmap=cmap, norm=norm)

  ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
  ax.set_xticks(np.arange(-.5, 3, 1));
  ax.set_yticks(np.arange(-.5, 3, 1));

  plt.show()

# Define AI Models

In [ ]:
from abc import ABC, abstractmethod
class ModelWrapper(ABC):
  """wrapper for all tic-tac-toe models"""
  def __init__(self):
    self.points = 0
  @abstractmethod
  def move(self, board):
    pass
  @abstractmethod
  def train(self, result):
    pass

In [ ]:
class ExampleModel(ModelWrapper):
  """dumb model that will always play at (0, 0)"""
  def move(self, board):
    return (0, 0)

# Minimax Implementation for Testing

In [ ]:
import copy
class Board:
  def __init__(self, board=[[0, 0, 0], [0, 0, 0], [0, 0, 0]]):
    self.board = board
  def move(self, pos, color):
    new_board = copy.deepcopy(self.board)
    new_board[pos[0]][pos[1]] = color
    return Board(new_board)
  def check_row(self, i):
    for j in range(3):
      if self.board[i][j] != self.board[i][0]:
        return 0
    return self.board[i][0]
  def check_col(self, j):
    for i in range(3):
      if self.board[i][j] != self.board[0][j]:
        return 0
    return self.board[0][j]
  def winner(self) -> int:
    for i in range(3):
      if self.check_row(i) > 0:
        return self.check_row(i)
      if self.check_col(i) > 0:
        return self.check_col(i)
    if self.board[0][0] != 0:
      works = True
      for i in range(3):
        if self.board[i][i] != self.board[0][0]:
          works = False
          break
      if works:
        return self.board[0][0]
    if self.board[0][2] != 0:
      works = True
      for i in range(3):
        if self.board[i][2 - i] != self.board[0][2]:
          works = False
          break
      if works:
        return self.board[0][2]
    for i in range(3):
      for j in range(3):
        if self.board[i][j] == 0:
          return 0
    return 3
  def __getitem__(self, index):
    return self.board[index]
  def __str__(self) -> str:
    return str(self.board)
  def __hash__(self):
    return hash(str(self.board))
  def __eq__(self, other):
    return self.board == other.board

In [ ]:
class Minimax(ModelWrapper):
  def __init__(self):
    self.cache = {}
    self.next_move = {}
    self.search(Board(), 1)
  def search(self, state: Board, color) -> int:
    # print(state)
    if state in self.cache:
      return self.cache[state]
    elif state.winner() > 0:
      # print(state)
      self.next_move[state] = (-1, -1)
      W = state.winner()
      if W == 3:
        self.cache[state] = 0
      elif W == color:
        self.cache[state] = 1
      else:
        self.cache[state] = -1
      return self.cache[state]
    else:
      mn = 2
      next_color = 1 if color == 2 else 2
      for i in range(3):
        for j in range(3):
          if state[i][j] != 0:
            continue
          next_state = state.move((i, j), color)
          if self.search(next_state, next_color) < mn:
            mn = self.cache[next_state]
            self.next_move[state] = (i, j)
      self.cache[state] = -mn
      return self.cache[state]
  def move(self, board):
    return self.next_move[Board(board)]
    # ^ to use this with the central host, uncomment the line above and comment the line below
    # return self.next_move[board]
  def train(self, result):
    pass

In [ ]:
minimax = Minimax()
minimax.move([[0, 0, 0], [0, 0, 0], [0, 0, 0]])

(0, 0)

# Controller

In [ ]:
class Controller:
  def __init__(self, verbose: bool):
    self.board = [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]
    ]
    self.verbose = verbose
  def play(self, move: tuple[int, int], color):
    """attempt to play move defined by tuple (x, y)"""
    if self.board[move[0]][move[1]] != 0:
      if self.verbose:
        print("Illegal move detected.")
        self.show_board()
        print(f"AI {color} attempted to play a move at {move}")
      return False
    else:
      self.board[move[0]][move[1]] = color
      if self.verbose:
        self.show_board()
      return True


  #This is the host function for checking if the game has ended
  def checkIfDone(self):
    """
    1->Player 1 wins
    2->Player 2 wins
    0->Draw
    -1->Unfinished
    """
    # check horizontal spaces
    for row in self.board:
        if row.count(row[0]) == len(row) and row[0] != 0:
            return row[0]

    # check vertical spaces
    for col in range(len(self.board[0])):
        check = []
        for row in self.board:
            check.append(row[col])
        if check.count(check[0]) == len(check) and check[0] != 0:
            return check[0]

    # check / diagonal spaces
    if self.board[0][0] != 0 and self.board[0][0] == self.board[1][1] == self.board[2][2]:
        return self.board[0][0]

    # check \ diagonal spaces
    if self.board[0][2] != 0 and self.board[0][2] == self.board[1][1] == self.board[2][0]:
        return self.board[0][2]

    # check if board is full (draw)
    if all(all(row) for row in self.board):
        return 0

    # if the game is not over, return -1
    return -1
  def show_board(self):
    pretty_print(self.board)

# Runner

In [ ]:
#Main runner. No need to change anything just run.
import copy
from enum import Enum
player1_points = 0
player2_points = 0
visible_buffer_games=1
invisible_games=10000
illegal = False
class Result(Enum):
    WIN = 1
    LOSS = 2
    DRAW = 3
    ILLEGAL = 4
def run_episode(verbose: bool, first, second, train=True)->Result:
  cnt = 0
  controller = Controller(verbose=verbose)
  # illegal = False
  # result = 0
  p1 = None
  p2 = None
  while controller.checkIfDone() == -1:
    first_move = first.move(controller.get_board())
    if controller.play(first_move, 1) == False:
      p1 = Result.ILLEGAL
      p2 = Result.WIN
      # result = -illegal_penalty
      break
    cnt += 1
    if controller.checkIfDone()== -1:
      second_move = second.move(controller.get_board())
      if controller.play(second_move, 2) == False:
        p1 = Result.WIN
        p2 = Result.ILLEGAL
        # result = illegal_penalty
        break
      cnt += 1
  if p1 == None:
    winner = controller.checkIfDone()
    if winner == 1:
      p1 = Result.WIN
      p2 = Result.LOSS
    elif winner == 2:
      p1 = Result.LOSS
      p2 = Result.WIN
    else:
      p1 = Result.DRAW
      p2 = Result.DRAW
    if verbose:
      controller.show_board()
  if p1 == Result.DRAW:
    first.points += 1
    second.points += 1
    if verbose:
      print("game was drawn")
  if p1 == Result.WIN:
    first.points += 2
    if verbose:
      print("player 1 won")
  elif p2 == Result.WIN:
    second.points += 2
    if verbose:
      print("player 2 won")
  if train:
    first.train(p1)
    second.train(p2)

print("Player 1 First Games:")
for x in range(visible_buffer_games):
  run_episode(verbose=True, first=player1, second=player2)

print("Invisible Games Playing...")

for x in range(invisible_games):
  run_episode(verbose=False, first=player1, second=player2)

for x in range(visible_buffer_games):
  run_episode(verbose=True, first=player1, second=player2)

print("Player 1 Points:", player1_points)
print("Player 2 Points:", player2_points)
print("Player 2 First Games:")
for x in range(visible_buffer_games):
  run_episode(verbose=True, first=player2, second=player1)

print("Invisible Games Playing...")

for x in range(invisible_games):
  run_episode(verbose=False, first=player2, second=player1)

for x in range(visible_buffer_games):
  run_episode(verbose=True, first=player2, second=player1)
print("Player 1 Points:", player1_points)
print("Player 2 Points:", player2_points)
print("Game Over")

Player 1 First Games:


NameError: ignored